In [2]:
from bs4 import BeautifulSoup

def parse_xml(path, type_data):
  with open(path, encoding='windows-1252') as fp:
    soup = BeautifulSoup(fp, 'html.parser')
  data = soup.find_all('lexelt')
  parsed_data = {}
  for word in data:
    new_data = []
    instances = word.find_all('instance')
    for inst in instances:
      sentence = inst.find('context').decode_contents().strip('\n')
      a, *b = sentence.split('<head>')
      b, *c = ''.join(b).split('</head>')
      sentence = a + b + ''.join(c)
      if type_data =="train":
        new_data.append((sentence, inst.find('answer').attrs['senseid']))
      else:
        new_data.append((sentence,))
    parsed_data[word.attrs["item"]] = new_data
     
  return parsed_data

test_labels = {}
labels = []
word0 = ""
with open("/content/drive/MyDrive/wsd/data_wsd/RomanianLS.test.key") as myfile:
    for ind, line in enumerate(myfile):
        if ind == 0:
          word0 = line.split(" ")[0].strip()
        sense = line.split(" ")[2].strip()
        word = line.split(" ")[0].strip()
        if word!=word0:
          test_labels[word0] = labels
          labels = []
          word0=word
        labels.append(sense)
test_labels[word0] = labels   
print(test_labels)
train_se3 = parse_xml("/content/drive/MyDrive/wsd/data_wsd/RomanianLS.train","train")
print(train_se3["accent.n"])
test_se3 = parse_xml("/content/drive/MyDrive/wsd/data_wsd/RomanianLS.test","test")
print(test_se3["accent.n"])


{'accent.n': ['a_pune_accent%01c', 'accent.n%02a', 'a_pune_accent%01c', 'accent.n%03a', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'accent.n%02a', 'a_pune_accent%01c', 'accent.n%02a', 'accent.n%02a', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'accent.n%02a', 'accent.n%02a', 'accent.n%02a', 'accent.n%02a', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'accent.n%02a', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'accent.n%02a', 'accent.n%01a', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'accent.n%02a', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', 'a_pune_accent%01c', '

In [6]:
!pip install spacy sklearn rowordnet
!python -m spacy download ro_core_news_lg
!pip install bs4
!pip install sparknlp pyspark
import nltk
import spacy
from sklearn.feature_extraction import DictVectorizer
import rowordnet as rwn
from nltk.classify import MaxentClassifier

vec = DictVectorizer()
wn = rwn.RoWordNet()
nlp = spacy.load("ro_core_news_lg")


def extract_hypernymes(word):
    hyper = []
    synset_ids = wn.synsets(literal=word.lemma_, pos=rwn.synset.Synset.Pos.NOUN)
    if len(synset_ids) > 0:
        c = wn.synset_to_hypernym_root(synset_ids[0])
        for c1 in c:
            if len(wn(c1).literals):
                hyper.append(wn(c1).literals[0])
    hyper_dict = {}
    for i in range(len(hyper)):
      hyper_dict["hyper"+str(i)] = hyper[i]
    #print(hyper_dict)
    return hyper_dict


def extract_feat_for_word(word, rel):
    hyper = {}
    if word.pos_ == "NOUN":
        hyper = extract_hypernymes(word)
    d = {
        ("text"+rel): word.text,
        "pos"+rel: word.pos_,
        "morpho"+rel: word.morph,
        "lemma"+rel: word.lemma_,
        "dep"+rel: word.dep_,
         "ner": word.ent_type_,
        #"hyper"+rel: hyper,
        #"position_rel": rel
    }
    d.update(hyper)
    return d

def create_features_vector(word, context):
    text = nlp(context)
    vect = {}
    for w in text:
      if w.lemma_ == word:
            if w.head != w:
              
              #print(w.head)
              vect.update(extract_feat_for_word(w.head, "head"))
              for ind, siblings in enumerate(w.head.children):
                vect.update(extract_feat_for_word(siblings, "0_"+str(ind) if w != siblings else "main"))
            else:
              vect.update(extract_feat_for_word(w, "main"))
            for chil in w.children:
              vect.update(extract_feat_for_word(chil, "children"))
        
    return vect

def prepare_data(data, data_type):
  new_data_per_words = {}

  for word, instances in data.items():
    sense = word.split('.')[0]
    new_data = []
    
    for ind, i in enumerate(instances):
      if data_type=="train":
        new_data.append((create_features_vector(sense,i[0]), i[1]))
      else:
        new_data.append((create_features_vector(sense,i[0]), test_labels[word][ind]))
    new_data_per_words[word] = new_data


  return new_data_per_words

a = prepare_data(train_se3,"train")

#test_data, test_labels = prepare_test_data(test_se3)
b = prepare_data(test_se3, "test")
print(a["accent.n"])
print(b["accent.n"])

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-19 08:17:47.462185: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.5/568.5 MB 2.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('ro_core_news_lg')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
[({}, 'a_pune_accent%01c'), ({'texthead': 'avut', 'poshead': 'VERB', 'morphohead': Gender=Masc|Number=Sing|VerbForm=

In [9]:

algorithm = nltk.classify.MaxentClassifier.ALGORITHMS[0]
classifiers = {}
for w in a.keys():
  classifier = nltk.MaxentClassifier.train(a[w], algorithm, max_iter=20)
  classifier.show_most_informative_features(10)
  classifiers[w] = classifier

for w in b.keys():
  #output_labels = classifiers[w].classify_many(b[w])
  print(w," ",nltk.classify.accuracy( classifiers[w], b[w]))


  ==> Training (20 iterations)

      Iteration    Log Likelihood    Accuracy
      ---------------------------------------
             1          -1.38629        0.017
             2          -0.84525        0.779
             3          -0.60904        0.837
             4          -0.48497        0.895
             5          -0.40865        0.919
             6          -0.35635        0.936
             7          -0.31774        0.948
             8          -0.28769        0.959
             9          -0.26340        0.965
            10          -0.24321        0.971
            11          -0.22608        0.971
            12          -0.21130        0.971
            13          -0.19841        0.971
            14          -0.18705        0.977
            15          -0.17696        0.977
            16          -0.16793        0.983
            17          -0.15979        0.988
            18          -0.15243        0.988
            19          -0.14573        0.988
  